In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Импорт инструментов для машинного обучения из scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             mean_absolute_error, mean_squared_error, r2_score,
                             confusion_matrix, classification_report)

Загрузка данных и предварительная обработка данных для классификации

In [62]:
cancer_data_url = "https://raw.githubusercontent.com/KaiserRed/AIFrameworks/main/data/Cancer_Data.csv"
df_cancer = pd.read_csv(cancer_data_url)

print("\nИнформация о данных:")
df_cancer.info()
df_cancer = df_cancer.drop(columns=['Unnamed: 32', 'id'])

# Проверим баланс целевого признака 'diagnosis'
print("\nРаспределение целевого признака 'diagnosis':")
print(df_cancer['diagnosis'].value_counts())
print(f"\nСоотношение классов: M - {sum(df_cancer['diagnosis'] == 'M') / len(df_cancer):.2%}, B - {sum(df_cancer['diagnosis'] == 'B') / len(df_cancer):.2%}")


Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  peri

Загрузка данных и предварительная обработка данных для регрессии

In [63]:
laptop_data_url = "https://raw.githubusercontent.com/KaiserRed/AIFrameworks/main/data/laptop_prices.csv"
df_laptop = pd.read_csv(laptop_data_url)

print("\nИнформация о данных:")
df_laptop.info()

# Проверим наличие пропущенных значений
print("Пропущенные значения в датасете Laptop:")
print(df_laptop.isnull().sum())

# Проверим целевую переменную 'Price_euros'
print("\nОписательная статистика целевой переменной 'Price_euros':")
print(df_laptop['Price_euros'].describe())



Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1275 non-null   object 
 1   Product               1275 non-null   object 
 2   TypeName              1275 non-null   object 
 3   Inches                1275 non-null   float64
 4   Ram                   1275 non-null   int64  
 5   OS                    1275 non-null   object 
 6   Weight                1275 non-null   float64
 7   Price_euros           1275 non-null   float64
 8   Screen                1275 non-null   object 
 9   ScreenW               1275 non-null   int64  
 10  ScreenH               1275 non-null   int64  
 11  Touchscreen           1275 non-null   object 
 12  IPSpanel              1275 non-null   object 
 13  RetinaDisplay         1275 non-null   object 
 14  CPU_company           1275 non-null   object 
 15 

Подготовка данных для классификации

In [64]:
# Разделим данные на признаки (X) и целевую переменную (y)
X_cancer = df_cancer.drop(columns=['diagnosis'])
y_cancer = df_cancer['diagnosis']

# Закодируем целевую переменную в числовой формат (M -> 1, B -> 0)
le = LabelEncoder()
y_cancer_encoded = le.fit_transform(y_cancer)
print(f"Закодированные значения: {le.classes_} -> {le.transform(le.classes_)}")

# Разделим данные на обучающую и тестовую выборки (80% / 20%)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_cancer, y_cancer_encoded, test_size=0.2, random_state=42, stratify=y_cancer_encoded
)
print(f"\nРазмеры выборок для классификации:")
print(f"Обучающая: {X_train_c.shape}, Тестовая: {X_test_c.shape}")

Закодированные значения: ['B' 'M'] -> [0 1]

Размеры выборок для классификации:
Обучающая: (455, 30), Тестовая: (114, 30)


Подготовка данных для регрессии

In [65]:
# Выделим целевую переменную и признаки
y_laptop = df_laptop['Price_euros']
X_laptop = df_laptop.drop(columns=['Price_euros'])

# Разделим данные на обучающую и тестовую выборки (80% / 20%)
X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(
    X_laptop, y_laptop, test_size=0.2, random_state=42
)
print(f"Размеры выборок для регрессии:")
print(f"Обучающая: {X_train_l.shape}, Тестовая: {X_test_l.shape}")

Размеры выборок для регрессии:
Обучающая: (1020, 22), Тестовая: (255, 22)


## Бейзлайн с решающим деревом


Для классификации

Создаем и обучаем решающее дерево для классификации с параметрами по умолчанию. Оцениваем качество с помощью основных метрик классификации.

In [66]:
# Создаем и обучаем модель решающего дерева для классификации с параметрами по умолчанию
base_tree_clf = DecisionTreeClassifier(random_state=42)
base_tree_clf.fit(X_train_c, y_train_c)

# Прогнозируем на тестовой выборке
y_pred_c_base = base_tree_clf.predict(X_test_c)

# Оцениваем качество модели
accuracy_base = accuracy_score(y_test_c, y_pred_c_base)
precision_base = precision_score(y_test_c, y_pred_c_base)
recall_base = recall_score(y_test_c, y_pred_c_base)
f1_base = f1_score(y_test_c, y_pred_c_base)

print("Бейзлайн модель (Decision Tree Classifier) - оценка качества:")
print(f"Accuracy: {accuracy_base:.4f}")
print(f"Precision: {precision_base:.4f}")
print(f"Recall: {recall_base:.4f}")
print(f"F1-score: {f1_base:.4f}")

# Матрица ошибок
cm_base = confusion_matrix(y_test_c, y_pred_c_base)
print("\nМатрица ошибок:")
print(cm_base)

Бейзлайн модель (Decision Tree Classifier) - оценка качества:
Accuracy: 0.9298
Precision: 0.9048
Recall: 0.9048
F1-score: 0.9048

Матрица ошибок:
[[68  4]
 [ 4 38]]


Для регрессии

Для регрессии необходимо обработать категориальные признаки с помощью OneHotEncoding. Обучаем решающее дерево для регрессии и оцениваем его с помощью метрик MAE, MSE, RMSE и R².

In [67]:
# Для регрессии нужно обработать категориальные признаки перед обучением
# Выделим числовые и категориальные признаки
num_cols_laptop = X_train_l.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_laptop = X_train_l.select_dtypes(include=['object']).columns.tolist()

print(f"Числовые признаки: {num_cols_laptop}")
print(f"Категориальные признаки: {cat_cols_laptop}")

# Применим OneHotEncoder к категориальным признакам
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_train_cat = ohe.fit_transform(X_train_l[cat_cols_laptop])
X_test_cat = ohe.transform(X_test_l[cat_cols_laptop])

# Объединим числовые и закодированные категориальные признаки
X_train_l_processed = np.hstack([X_train_l[num_cols_laptop].values, X_train_cat])
X_test_l_processed = np.hstack([X_test_l[num_cols_laptop].values, X_test_cat])

print(f"\nРазмерности после обработки категориальных признаков:")
print(f"Обучающая: {X_train_l_processed.shape}, Тестовая: {X_test_l_processed.shape}")

# Создаем и обучаем модель решающего дерева для регрессии с параметрами по умолчанию
base_tree_reg = DecisionTreeRegressor(random_state=42)
base_tree_reg.fit(X_train_l_processed, y_train_l)

# Прогнозируем на тестовой выборке
y_pred_l_base = base_tree_reg.predict(X_test_l_processed)

# Оцениваем качество модели
mae_base = mean_absolute_error(y_test_l, y_pred_l_base)
mse_base = mean_squared_error(y_test_l, y_pred_l_base)
rmse_base = np.sqrt(mse_base)
r2_base = r2_score(y_test_l, y_pred_l_base)

print("\nБейзлайн модель (Decision Tree Regressor) - оценка качества:")
print(f"MAE: {mae_base:.2f}")
print(f"MSE: {mse_base:.2f}")
print(f"RMSE: {rmse_base:.2f}")
print(f"R2: {r2_base:.4f}")

Числовые признаки: ['Inches', 'Ram', 'Weight', 'ScreenW', 'ScreenH', 'CPU_freq', 'PrimaryStorage', 'SecondaryStorage']
Категориальные признаки: ['Company', 'Product', 'TypeName', 'OS', 'Screen', 'Touchscreen', 'IPSpanel', 'RetinaDisplay', 'CPU_company', 'CPU_model', 'PrimaryStorageType', 'SecondaryStorageType', 'GPU_company', 'GPU_model']

Размерности после обработки категориальных признаков:
Обучающая: (1020, 779), Тестовая: (255, 779)

Бейзлайн модель (Decision Tree Regressor) - оценка качества:
MAE: 234.67
MSE: 118405.51
RMSE: 344.10
R2: 0.7614


### Улучшение бейзлайна

#### Гипотезы

Для классификации:
- Гипотеза 1: Масштабирование признаков улучшит качество дерева (хотя деревья не чувствительны к масштабированию, проверим).

- Гипотеза 2: Подбор гиперпараметров (глубина дерева, минимальное число образцов в листе) на кросс-валидации улучшит качество и уменьшит переобучение.

- Гипотеза 3: Использование критерия "энтропия" вместо "джини" может дать лучшие результаты.

Для регрессии:

- Гипотеза 1: Логарифмирование целевой переменной (цены) улучшит качество, так как распределение цены скошено.

- Гипотеза 2: Подбор гиперпараметров (глубина, минимальное число образцов) улучшит модель.

- Гипотеза 3: Удаление выбросов в целевой переменной улучшит стабильность модели.

Классификация

In [68]:
# Гипотеза 1: Масштабирование признаков
scaler = StandardScaler()
X_train_c_scaled = scaler.fit_transform(X_train_c)
X_test_c_scaled = scaler.transform(X_test_c)

tree_clf_scaled = DecisionTreeClassifier(random_state=42)
tree_clf_scaled.fit(X_train_c_scaled, y_train_c)
y_pred_c_scaled = tree_clf_scaled.predict(X_test_c_scaled)

print("Гипотеза 1 (масштабирование):")
print(f"Accuracy: {accuracy_score(y_test_c, y_pred_c_scaled):.4f}")
print(f"F1-score: {f1_score(y_test_c, y_pred_c_scaled):.4f}")

Гипотеза 1 (масштабирование):
Accuracy: 0.9298
F1-score: 0.9048


In [69]:
# Гипотеза 2 и 3: Подбор гиперпараметров
param_grid_clf = {
    'max_depth': [3, 5, 7, 10, 15, None],
    'min_samples_leaf': [1, 2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_clf = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_clf,
    cv=5,
    scoring='f1',
    n_jobs=-1
)
grid_clf.fit(X_train_c, y_train_c)

print("Лучшие параметры для классификации:")
print(grid_clf.best_params_)
print(f"Лучший F1-score на кросс-валидации: {grid_clf.best_score_:.4f}")

# Оценка на тестовой выборке
best_tree_clf = grid_clf.best_estimator_
y_pred_c_best = best_tree_clf.predict(X_test_c)

print("\nРезультаты улучшенной модели на тестовой выборке:")
print(f"Accuracy: {accuracy_score(y_test_c, y_pred_c_best):.4f}")
print(f"Precision: {precision_score(y_test_c, y_pred_c_best):.4f}")
print(f"Recall: {recall_score(y_test_c, y_pred_c_best):.4f}")
print(f"F1-score: {f1_score(y_test_c, y_pred_c_best):.4f}")

Лучшие параметры для классификации:
{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 1}
Лучший F1-score на кросс-валидации: 0.9227

Результаты улучшенной модели на тестовой выборке:
Accuracy: 0.9386
Precision: 0.9487
Recall: 0.8810
F1-score: 0.9136


Регрессия

In [70]:
# Гипотеза 1: Логарифмирование целевой переменной
y_train_l_log = np.log1p(y_train_l)
y_test_l_log = np.log1p(y_test_l)

tree_reg_log = DecisionTreeRegressor(random_state=42)
tree_reg_log.fit(X_train_l_processed, y_train_l_log)
y_pred_l_log = tree_reg_log.predict(X_test_l_processed)

# Обратное преобразование для оценки в исходной шкале
y_pred_l_exp = np.expm1(y_pred_l_log)

print("Гипотеза 1 (логарифмирование целевой переменной):")
print(f"MAE: {mean_absolute_error(y_test_l, y_pred_l_exp):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_l, y_pred_l_exp)):.2f}")
print(f"R2: {r2_score(y_test_l, y_pred_l_exp):.4f}")

Гипотеза 1 (логарифмирование целевой переменной):
MAE: 203.07
RMSE: 294.28
R2: 0.8255


In [71]:
# Гипотеза 2: Подбор гиперпараметров для регрессии
# Используем кросс-валидацию и метрику RMSE.
param_grid_reg = {
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_leaf': [1, 2, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_reg = GridSearchCV(
    DecisionTreeRegressor(random_state=42),
    param_grid_reg,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1
)
grid_reg.fit(X_train_l_processed, y_train_l)

print("Лучшие параметры для регрессии:")
print(grid_reg.best_params_)
print(f"Лучший RMSE на кросс-валидации: {-grid_reg.best_score_:.2f}")

# Оценка на тестовой выборке
best_tree_reg = grid_reg.best_estimator_
y_pred_l_best = best_tree_reg.predict(X_test_l_processed)

print("\nРезультаты улучшенной модели на тестовой выборке:")
print(f"MAE: {mean_absolute_error(y_test_l, y_pred_l_best):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_l, y_pred_l_best)):.2f}")
print(f"R2: {r2_score(y_test_l, y_pred_l_best):.4f}")

Лучшие параметры для регрессии:
{'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 2}
Лучший RMSE на кросс-валидации: 347.26

Результаты улучшенной модели на тестовой выборке:
MAE: 242.31
RMSE: 363.26
R2: 0.7341


In [72]:
# Гипотеза 3: Удаление выбросов в целевой переменной
# Определим выбросы с помощью межквартильного размаха (IQR)
Q1 = y_train_l.quantile(0.25)
Q3 = y_train_l.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Создаем маску для невыбросных значений
no_outliers_mask = (y_train_l >= lower_bound) & (y_train_l <= upper_bound)

print(f"Количество выбросов в обучающей выборке: {sum(~no_outliers_mask)}")
print(f"Процент выбросов: {sum(~no_outliers_mask) / len(y_train_l):.2%}")

# Обучаем на данных без выбросов
X_train_l_no_out = X_train_l_processed[no_outliers_mask]
y_train_l_no_out = y_train_l[no_outliers_mask]

tree_reg_no_out = DecisionTreeRegressor(random_state=42, max_depth=10, min_samples_leaf=5)
tree_reg_no_out.fit(X_train_l_no_out, y_train_l_no_out)
y_pred_l_no_out = tree_reg_no_out.predict(X_test_l_processed)

print("\nГипотеза 3 (удаление выбросов):")
print(f"MAE: {mean_absolute_error(y_test_l, y_pred_l_no_out):.2f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test_l, y_pred_l_no_out)):.2f}")
print(f"R2: {r2_score(y_test_l, y_pred_l_no_out):.4f}")

Количество выбросов в обучающей выборке: 22
Процент выбросов: 2.16%

Гипотеза 3 (удаление выбросов):
MAE: 243.93
RMSE: 428.70
R2: 0.6297


Сравнение результаты гипотез для классификации

In [73]:
# Сводная таблица результатов классификации
results_clf = {
    'Модель': ['Бейзлайн', 'Со scaling', 'Улучшенная (GridSearch)'],
    'Accuracy': [accuracy_base, accuracy_score(y_test_c, y_pred_c_scaled), accuracy_score(y_test_c, y_pred_c_best)],
    'F1-score': [f1_base, f1_score(y_test_c, y_pred_c_scaled), f1_score(y_test_c, y_pred_c_best)]
}

df_results_clf = pd.DataFrame(results_clf)
print("Сравнение моделей классификации:")
display(df_results_clf)



Сравнение моделей классификации:


,Модель,Accuracy,F1-score
0,Бейзлайн,0.929825,0.904762
1,Со scaling,0.929825,0.904762
2,Улучшенная (GridSearch),0.938596,0.913580


Сравнение результаты гипотез для регрессии

In [74]:
# Сводная таблица результатов регрессии
results_reg = {
    'Модель': ['Бейзлайн', 'Логарифмирование', 'GridSearch', 'Без выбросов'],
    'MAE': [mae_base, mean_absolute_error(y_test_l, y_pred_l_exp),
            mean_absolute_error(y_test_l, y_pred_l_best),
            mean_absolute_error(y_test_l, y_pred_l_no_out)],
    'RMSE': [rmse_base, np.sqrt(mean_squared_error(y_test_l, y_pred_l_exp)),
             np.sqrt(mean_squared_error(y_test_l, y_pred_l_best)),
             np.sqrt(mean_squared_error(y_test_l, y_pred_l_no_out))],
    'R2': [r2_base, r2_score(y_test_l, y_pred_l_exp),
           r2_score(y_test_l, y_pred_l_best),
           r2_score(y_test_l, y_pred_l_no_out)]
}

df_results_reg = pd.DataFrame(results_reg)
print("Сравнение моделей регрессии:")
display(df_results_reg)

Сравнение моделей регрессии:


,Модель,MAE,RMSE,R2
0,Бейзлайн,234.667922,344.101020,0.761442
1,Логарифмирование,203.067053,294.280438,0.825521
2,GridSearch,242.314774,363.256133,0.734144
3,Без выбросов,243.926989,428.702852,0.629717


### Формирование улучшенного бейзлайна на основе проверенных гипотез
Для классификации:

- Масштабирование не требуется (деревья инвариантны)
- Используем лучшие параметры: max_depth=7, min_samples_leaf=1, criterion=gini
-  Критерий gini работает лучше чем entropy

Для регрессии:

-  Применяем логарифмирование целевой переменной"
-  Используем параметры по умолчанию (подбор без логарифмирования не улучшает)
-  Не удаляем выбросы
-  Используем все признаки (категориальные через OneHotEncoding)

##  Обучение моделей с улучшенным бейзлайном
Улучшенный бейзлайн для классификации

Используем лучшие найденные параметры. Анализируем важность признаков в улучшенной модели.

In [75]:
# Создаем улучшенную модель на основе проверенных гипотез
improved_tree_clf = DecisionTreeClassifier(
    max_depth=7,
    min_samples_leaf=1,
    criterion='gini',
    random_state=42
)

# Обучаем на тех же данных (масштабирование не требуется)
improved_tree_clf.fit(X_train_c, y_train_c)

# Предсказания
y_pred_improved_clf = improved_tree_clf.predict(X_test_c)

# Оценка качества улучшенной модели
accuracy_improved_clf = accuracy_score(y_test_c, y_pred_improved_clf)
precision_improved_clf = precision_score(y_test_c, y_pred_improved_clf)
recall_improved_clf = recall_score(y_test_c, y_pred_improved_clf)
f1_improved_clf = f1_score(y_test_c, y_pred_improved_clf)

print("Оценка качества улучшенной модели классификации:")
print(f"Accuracy: {accuracy_improved_clf:.4f}")
print(f"Precision: {precision_improved_clf:.4f}")
print(f"Recall: {recall_improved_clf:.4f}")
print(f"F1-score: {f1_improved_clf:.4f}")

# Матрица ошибок
cm_improved = confusion_matrix(y_test_c, y_pred_improved_clf)
print("\nМатрица ошибок улучшенной модели:")
print(cm_improved)

# Визуализация важности признаков
feature_importance_clf = pd.DataFrame({
    'feature': X_train_c.columns,
    'importance': improved_tree_clf.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 самых важных признаков:")
display(feature_importance_clf.head(10))

Оценка качества улучшенной модели классификации:
Accuracy: 0.9386
Precision: 0.9487
Recall: 0.8810
F1-score: 0.9136

Матрица ошибок улучшенной модели:
[[70  2]
 [ 5 37]]

Топ-10 самых важных признаков:


,feature,importance
22,perimeter_worst,0.737370
27,concave points_worst,0.068621
1,texture_mean,0.055837
24,smoothness_worst,0.041388
3,area_mean,0.022758
23,area_worst,0.015819
4,smoothness_mean,0.011423
2,perimeter_mean,0.011423
21,texture_worst,0.010160
18,symmetry_se,0.007139


 Улучшенный бейзлайн для регрессии.

 Используем все признаки с OneHotEncoding

In [76]:
# Применяем логарифмирование целевой переменной (лучшая техника)
y_train_l_log = np.log1p(y_train_l)
y_test_l_log = np.log1p(y_test_l)

# Создаем улучшенную модель с параметрами по умолчанию
# (подбор параметров без логарифмирования не дал улучшения)
improved_tree_reg = DecisionTreeRegressor(random_state=42)

# Обучаем на логарифмированных данных
improved_tree_reg.fit(X_train_l_processed, y_train_l_log)

# Предсказания (в логарифмированной шкале)
y_pred_log_improved = improved_tree_reg.predict(X_test_l_processed)

# Обратное преобразование для оценки в исходной шкале
y_pred_improved_reg = np.expm1(y_pred_log_improved)

# Оценка качества улучшенной модели
mae_improved_reg = mean_absolute_error(y_test_l, y_pred_improved_reg)
mse_improved_reg = mean_squared_error(y_test_l, y_pred_improved_reg)
rmse_improved_reg = np.sqrt(mse_improved_reg)
r2_improved_reg = r2_score(y_test_l, y_pred_improved_reg)

print("Оценка качества улучшенной модели регрессии:")
print(f"MAE: {mae_improved_reg:.2f}")
print(f"MSE: {mse_improved_reg:.2f}")
print(f"RMSE: {rmse_improved_reg:.2f}")
print(f"R2: {r2_improved_reg:.4f}")

# Визуализация важности признаков
feature_importance_reg = pd.DataFrame({
    'feature': list(num_cols_laptop) + list(ohe.get_feature_names_out()),
    'importance': improved_tree_reg.feature_importances_
}).sort_values('importance', ascending=False)

print("\nТоп-10 самых важных признаков:")
display(feature_importance_reg.head(10))

Оценка качества улучшенной модели регрессии:
MAE: 203.07
MSE: 86600.98
RMSE: 294.28
R2: 0.8255

Топ-10 самых важных признаков:


,feature,importance
1,Ram,0.595247
5,CPU_freq,0.074800
554,TypeName_Notebook,0.052428
2,Weight,0.035166
0,Inches,0.034905
569,Screen_Standard,0.025279
6,PrimaryStorage,0.018928
629,CPU_model_Core i5 8250U,0.008500
561,OS_No OS,0.007452
612,CPU_model_Core i3 6006U,0.007409


Сравниваем метрики исходного и улучшенного бейзлайнов

In [77]:
# Создаем сравнительные таблицы
print("\nКЛАССИФИКАЦИЯ (Рак молочной железы):")
print("-" * 60)

comparison_clf = pd.DataFrame({
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
    'Исходный бейзлайн': [accuracy_base, precision_base, recall_base, f1_base],
    'Улучшенный бейзлайн': [accuracy_improved_clf, precision_improved_clf,
                           recall_improved_clf, f1_improved_clf],
    'Изменение': [accuracy_improved_clf - accuracy_base,
                  precision_improved_clf - precision_base,
                  recall_improved_clf - recall_base,
                  f1_improved_clf - f1_base]
})

# Форматируем для лучшего отображения
comparison_clf_display = comparison_clf.copy()
for col in ['Исходный бейзлайн', 'Улучшенный бейзлайн', 'Изменение']:
    comparison_clf_display[col] = comparison_clf_display[col].apply(lambda x: f"{x:.4f}")

display(comparison_clf_display)

print("\nРЕГРЕССИЯ (Цены ноутбуков):")
print("-" * 60)

comparison_reg = pd.DataFrame({
    'Метрика': ['MAE', 'RMSE', 'R²'],
    'Исходный бейзлайн': [mae_base, rmse_base, r2_base],
    'Улучшенный бейзлайн': [mae_improved_reg, rmse_improved_reg, r2_improved_reg],
    'Изменение': [mae_improved_reg - mae_base,
                  rmse_improved_reg - rmse_base,
                  r2_improved_reg - r2_base]
})

# Для метрик ошибок (MAE, RMSE) отрицательное изменение - улучшение
# Для R² положительное изменение - улучшение
comparison_reg_display = comparison_reg.copy()
for col in ['Исходный бейзлайн', 'Улучшенный бейзлайн']:
    if col in ['MAE', 'RMSE']:
        comparison_reg_display[col] = comparison_reg_display[col].apply(lambda x: f"{x:.2f}")
    else:
        comparison_reg_display[col] = comparison_reg_display[col].apply(lambda x: f"{x:.4f}")

comparison_reg_display['Изменение'] = comparison_reg['Изменение'].apply(lambda x: f"{x:+.2f}" if abs(x) >= 0.01 else f"{x:+.4f}")

display(comparison_reg_display)




КЛАССИФИКАЦИЯ (Рак молочной железы):
------------------------------------------------------------


,Метрика,Исходный бейзлайн,Улучшенный бейзлайн,Изменение
0,Accuracy,0.9298,0.9386,0.0088
1,Precision,0.9048,0.9487,0.0440
2,Recall,0.9048,0.8810,-0.0238
3,F1-score,0.9048,0.9136,0.0088



РЕГРЕССИЯ (Цены ноутбуков):
------------------------------------------------------------


,Метрика,Исходный бейзлайн,Улучшенный бейзлайн,Изменение
0,MAE,234.6679,203.0671,-31.60
1,RMSE,344.1010,294.2804,-49.82
2,R²,0.7614,0.8255,+0.06


## Имплементация решающего дерева

АЛГОРИТМ ПОСТРОЕНИЯ

Основные шаги:

- Выбрать лучший признак и порог для разделения

- Разделить данные на две подгруппы

- Рекурсивно повторить для каждой подгруппы

- Остановиться при выполнении критерия остановки

КРИТЕРИИ ОСТАНОВКИ
- Максимальная глубина дерева достигнута

- Минимальное количество образцов в узле для разделения

- Минимальное количество образцов в листе

- Нет улучшения в критерии разделения

- Все образцы принадлежат одному классу (для классификации)

- Дисперсия целевой переменной ниже порога (для регрессии)


#### Для классификации

Используем критерий Джини для определения лучшего разделения


In [84]:
class DetailedDecisionTreeClassifier:
    """
    Подробная реализация решающего дерева для классификации
    с поддержкой критериев Джини и энтропии
    """

    def __init__(self,
                 criterion='gini',
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 min_impurity_decrease=0.0,
                 max_features=None,
                 random_state=None):

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.max_features = max_features
        self.random_state = random_state
        self.tree = None
        self.n_classes = None
        self.n_features = None
        self.feature_importances_ = None

        if random_state is not None:
            np.random.seed(random_state)

    def _gini(self, y):
        """Вычисляет коэффициент Джини для набора меток."""
        if len(y) == 0:
            return 0.0

        # Получаем количество каждого класса
        unique, counts = np.unique(y, return_counts=True)
        proportions = counts / len(y)
        gini = 1.0 - np.sum(proportions ** 2)

        return gini

    def _entropy(self, y):
        """Вычисляет энтропию для набора меток."""
        if len(y) == 0:
            return 0.0

        # Получаем количество каждого класса
        unique, counts = np.unique(y, return_counts=True)
        proportions = counts / len(y)

        # Энтропия: -Σ p_i * log2(p_i)
        # Добавляем малую величину чтобы избежать log(0)
        entropy = -np.sum(proportions * np.log2(proportions + 1e-10))

        return entropy

    def _impurity(self, y):
        """Вычисляет неоднородность в зависимости от выбранного критерия."""
        if self.criterion == 'gini':
            return self._gini(y)
        elif self.criterion == 'entropy':
            return self._entropy(y)
        else:
            raise ValueError(f"Unknown criterion: {self.criterion}")

    def _find_best_split(self, X, y, feature_indices):
        """
        Находит лучшее разделение для заданных данных и признаков.

        Параметры:
        ----------
        X : ndarray, форма (n_samples, n_features)
            Матрица признаков
        y : ndarray, форма (n_samples,)
            Вектор целевых значений
        feature_indices : list
            Индексы признаков для рассмотрения

        Возвращает:
        -----------
        best_feature : int or None
            Индекс лучшего признака
        best_threshold : float or None
            Лучший порог разделения
        best_impurity_decrease : float
            Уменьшение неоднородности при лучшем разделении
        """
        n_samples = X.shape[0]
        parent_impurity = self._impurity(y)
        best_impurity_decrease = -float('inf')
        best_feature = None
        best_threshold = None

        # Если нет достаточного количества образцов или чистота уже высокая
        if n_samples < self.min_samples_split or parent_impurity < 1e-10:
            return None, None, 0.0

        # Перебираем признаки
        for feature_idx in feature_indices:
            # Получаем уникальные значения признака
            feature_values = X[:, feature_idx]
            unique_values = np.unique(feature_values)

            # Если значений мало, проверяем все
            if len(unique_values) <= 1:
                continue

            # Сортируем уникальные значения
            unique_values.sort()

            # Рассматриваем пороги как средние между соседними значениями
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2.0

            # Для каждого порога проверяем разделение
            for threshold in thresholds:
                # Разделяем данные
                left_mask = feature_values <= threshold
                right_mask = ~left_mask

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)

                # Проверяем минимальное количество образцов в листьях
                if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                    continue

                # Вычисляем взвешенную неоднородность
                impurity_left = self._impurity(y[left_mask])
                impurity_right = self._impurity(y[right_mask])

                weighted_impurity = (n_left / n_samples) * impurity_left + \
                                   (n_right / n_samples) * impurity_right

                # Вычисляем уменьшение неоднородности
                impurity_decrease = parent_impurity - weighted_impurity

                # Проверяем минимальное уменьшение неоднородности
                if impurity_decrease < self.min_impurity_decrease:
                    continue

                # Обновляем лучшее разделение
                if impurity_decrease > best_impurity_decrease:
                    best_impurity_decrease = impurity_decrease
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold, best_impurity_decrease

    def _get_feature_indices(self, n_features):
        """Выбирает признаки для рассмотрения."""
        if self.max_features is None:
            # Все признаки
            return list(range(n_features))
        elif isinstance(self.max_features, int):
            # Случайный выбор фиксированного количества признаков
            return np.random.choice(n_features, self.max_features, replace=False).tolist()
        elif isinstance(self.max_features, float):
            # Случайный выбор доли признаков
            n = max(1, int(self.max_features * n_features))
            return np.random.choice(n_features, n, replace=False).tolist()
        else:
            raise ValueError(f"Invalid max_features: {self.max_features}")

    def _build_tree(self, X, y, depth=0):
        """
        Рекурсивно строит дерево решений.

        Параметры:
        ----------
        X : ndarray
            Матрица признаков
        y : ndarray
            Вектор целевых значений
        depth : int
            Текущая глубина дерева

        Возвращает:
        -----------
        node : dict or int
            Узел дерева или лист (предсказанный класс)
        """
        n_samples, n_features = X.shape

        # Вычисляем распределение классов
        unique_classes, class_counts = np.unique(y, return_counts=True)
        majority_class = unique_classes[np.argmax(class_counts)]

        # Критерии остановки
        stop_conditions = [
            # 1. Достигнута максимальная глубина
            (self.max_depth is not None and depth >= self.max_depth),

            # 2. Недостаточно образцов для разделения
            n_samples < self.min_samples_split,

            # 3. Все образцы одного класса
            len(unique_classes) == 1,

            # 4. Недостаточно признаков для разделения
            n_features == 0,
        ]

        if any(stop_conditions):
            # Создаем лист
            leaf_value = {
                'value': majority_class,
                'proba': class_counts / n_samples,
                'n_samples': n_samples,
                'is_leaf': True
            }
            return leaf_value

        # Выбираем признаки для рассмотрения
        feature_indices = self._get_feature_indices(n_features)

        # Ищем лучшее разделение
        best_feature, best_threshold, impurity_decrease = self._find_best_split(
            X, y, feature_indices
        )

        # Если не нашли подходящего разделения
        if best_feature is None or impurity_decrease <= 0:
            leaf_value = {
                'value': majority_class,
                'proba': class_counts / n_samples,
                'n_samples': n_samples,
                'is_leaf': True
            }
            return leaf_value

        # Разделяем данные
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        # Рекурсивно строим поддеревья
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        # Создаем узел
        node = {
            'feature_idx': best_feature,
            'threshold': best_threshold,
            'impurity_decrease': impurity_decrease,
            'n_samples': n_samples,
            'left': left_subtree,
            'right': right_subtree,
            'value': majority_class,
            'proba': class_counts / n_samples,
            'is_leaf': False
        }

        return node

    def _compute_feature_importances(self, node):
        """Вычисляет важность признаков."""
        if node is None:
            return

        if not node.get('is_leaf', True):
            # Узел вносит вклад в важность признака
            feature_idx = node['feature_idx']
            importance = (node['n_samples'] / self.n_samples_total) * node['impurity_decrease']

            self.feature_importances_[feature_idx] += importance

            # Рекурсивно обрабатываем дочерние узлы
            self._compute_feature_importances(node['left'])
            self._compute_feature_importances(node['right'])

    def fit(self, X, y):
        """
        Обучает модель на данных.

        Параметры:
        ----------
        X : array-like, форма (n_samples, n_features)
            Матрица признаков
        y : array-like, форма (n_samples,)
            Вектор целевых значений
        """
        X = np.array(X)
        y = np.array(y)

        self.n_samples_total = X.shape[0]
        self.n_features = X.shape[1]
        self.n_classes = len(np.unique(y))

        # Инициализируем важность признаков
        self.feature_importances_ = np.zeros(self.n_features)

        # Строим дерево
        self.tree = self._build_tree(X, y)

        # Вычисляем важность признаков
        self._compute_feature_importances(self.tree)

        # Нормализуем важность признаков
        if np.sum(self.feature_importances_) > 0:
            self.feature_importances_ /= np.sum(self.feature_importances_)

        return self

    def _predict_one(self, x, node):
        """Предсказывает класс для одного образца."""
        if node.get('is_leaf', True):
            return node['value']

        if x[node['feature_idx']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def _predict_proba_one(self, x, node):
        """Предсказывает вероятности классов для одного образца."""
        if node.get('is_leaf', True):
            # Создаем полный вектор вероятностей
            proba = np.zeros(self.n_classes)
            proba[node['value']] = 1.0
            return proba

        if x[node['feature_idx']] <= node['threshold']:
            return self._predict_proba_one(x, node['left'])
        else:
            return self._predict_proba_one(x, node['right'])

    def predict(self, X):
        """
        Предсказывает классы для набора данных.

        Параметры:
        ----------
        X : array-like, форма (n_samples, n_features)
            Матрица признаков

        Возвращает:
        -----------
        y_pred : ndarray, форма (n_samples,)
            Предсказанные классы
        """
        X = np.array(X)
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples, dtype=int)

        for i in range(n_samples):
            predictions[i] = self._predict_one(X[i], self.tree)

        return predictions

    def predict_proba(self, X):
        """
        Предсказывает вероятности классов.

        Параметры:
        ----------
        X : array-like, форма (n_samples, n_features)
            Матрица признаков

        Возвращает:
        -----------
        proba : ndarray, форма (n_samples, n_classes)
            Вероятности классов
        """
        X = np.array(X)
        n_samples = X.shape[0]
        proba = np.zeros((n_samples, self.n_classes))

        for i in range(n_samples):
            proba[i] = self._predict_proba_one(X[i], self.tree)

        return proba

    def get_depth(self, node=None):
        """Возвращает глубину дерева."""
        if node is None:
            node = self.tree

        if node.get('is_leaf', True):
            return 0

        left_depth = self.get_depth(node['left'])
        right_depth = self.get_depth(node['right'])

        return max(left_depth, right_depth) + 1

    def get_n_leaves(self, node=None):
        """Возвращает количество листьев в дереве."""
        if node is None:
            node = self.tree

        if node.get('is_leaf', True):
            return 1

        left_leaves = self.get_n_leaves(node['left'])
        right_leaves = self.get_n_leaves(node['right'])

        return left_leaves + right_leaves

# Тестируем детальную реализацию для классификации
print("\nТЕСТИРОВАНИЕ РЕАЛИЗАЦИИ ДЛЯ КЛАССИФИКАЦИИ")
print("-" * 60)

# Создаем модель с параметрами как у sklearn бейзлайна
detailed_tree_clf = DetailedDecisionTreeClassifier(
    criterion='gini',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

# Обучаем
detailed_tree_clf.fit(X_train_c.values, y_train_c)

# Предсказания
y_pred_detailed_clf = detailed_tree_clf.predict(X_test_c.values)

# Оценка качества
accuracy_detailed_clf = accuracy_score(y_test_c, y_pred_detailed_clf)
precision_detailed_clf = precision_score(y_test_c, y_pred_detailed_clf)
recall_detailed_clf = recall_score(y_test_c, y_pred_detailed_clf)
f1_detailed_clf = f1_score(y_test_c, y_pred_detailed_clf)

print("Качество детальной реализации Decision Tree Classifier:")
print(f"Accuracy:  {accuracy_detailed_clf:.4f}")
print(f"Precision: {precision_detailed_clf:.4f}")
print(f"Recall:    {recall_detailed_clf:.4f}")
print(f"F1-score:  {f1_detailed_clf:.4f}")

# Дополнительная информация о дереве
print(f"\nХарактеристики дерева:")
print(f"Глубина: {detailed_tree_clf.get_depth()}")
print(f"Количество листьев: {detailed_tree_clf.get_n_leaves()}")

# Проверяем важность признаков
print(f"\nТоп-5 самых важных признаков:")
feature_importance_df = pd.DataFrame({
    'Признак': X_train_c.columns,
    'Важность': detailed_tree_clf.feature_importances_
}).sort_values('Важность', ascending=False)

display(feature_importance_df.head(5))



ТЕСТИРОВАНИЕ РЕАЛИЗАЦИИ ДЛЯ КЛАССИФИКАЦИИ
------------------------------------------------------------
Качество детальной реализации Decision Tree Classifier:
Accuracy:  0.9298
Precision: 0.9250
Recall:    0.8810
F1-score:  0.9024

Характеристики дерева:
Глубина: 8
Количество листьев: 24

Топ-5 самых важных признаков:


,Признак,Важность
22,perimeter_worst,0.723157
1,texture_mean,0.076330
27,concave points_worst,0.067700
24,smoothness_worst,0.040833
3,area_mean,0.022452


Для регрессии

Формулы:
- MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y})^2
- MSE_{split} = \frac{n_{left}}{n_{total}} \cdot MSE_{left} + \frac{n_{right}}{n_{total}} \cdot MSE_{right}



In [85]:
class DetailedDecisionTreeRegressor:
    """
    Подробная реализация решающего дерева для регрессии
    """

    def __init__(self,
                 criterion='mse',
                 max_depth=None,
                 min_samples_split=2,
                 min_samples_leaf=1,
                 min_impurity_decrease=0.0,
                 max_features=None,
                 random_state=None):

        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.min_impurity_decrease = min_impurity_decrease
        self.max_features = max_features
        self.random_state = random_state
        self.tree = None
        self.n_features = None
        self.feature_importances_ = None

        if random_state is not None:
            np.random.seed(random_state)

    def _mse(self, y):
        """Вычисляет среднеквадратичную ошибку."""
        if len(y) == 0:
            return 0.0
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _mae(self, y):
        """Вычисляет среднюю абсолютную ошибку."""
        if len(y) == 0:
            return 0.0
        median = np.median(y)
        return np.mean(np.abs(y - median))

    def _impurity(self, y):
        """Вычисляет неоднородность в зависимости от критерия."""
        if self.criterion == 'mse':
            return self._mse(y)
        elif self.criterion == 'mae':
            return self._mae(y)
        elif self.criterion == 'friedman_mse':
            # Критерий Фридмана для MSE
            return self._mse(y)
        else:
            raise ValueError(f"Unknown criterion: {self.criterion}")

    def _find_best_split(self, X, y, feature_indices):
        """
        Находит лучшее разделение для регрессии.

        Параметры:
        ----------
        X : ndarray
            Матрица признаков
        y : ndarray
            Вектор целевых значений
        feature_indices : list
            Индексы признаков для рассмотрения

        Возвращает:
        -----------
        best_feature : int or None
            Индекс лучшего признака
        best_threshold : float or None
            Лучший порог разделения
        best_impurity_decrease : float
            Уменьшение неоднородности
        """
        n_samples = X.shape[0]
        parent_impurity = self._impurity(y)
        best_impurity_decrease = -float('inf')
        best_feature = None
        best_threshold = None

        # Проверка критериев остановки
        if n_samples < self.min_samples_split:
            return None, None, 0.0

        # Перебираем признаки
        for feature_idx in feature_indices:
            feature_values = X[:, feature_idx]
            unique_values = np.unique(feature_values)

            if len(unique_values) <= 1:
                continue

            # Сортируем значения
            unique_values.sort()

            # Рассматриваем пороги как средние между соседними значениями
            thresholds = (unique_values[:-1] + unique_values[1:]) / 2.0

            # Для каждого порога проверяем разделение
            for threshold in thresholds:
                left_mask = feature_values <= threshold
                right_mask = ~left_mask

                n_left = np.sum(left_mask)
                n_right = np.sum(right_mask)

                # Проверяем минимальное количество образцов
                if n_left < self.min_samples_leaf or n_right < self.min_samples_leaf:
                    continue

                # Вычисляем взвешенную неоднородность
                impurity_left = self._impurity(y[left_mask])
                impurity_right = self._impurity(y[right_mask])

                weighted_impurity = (n_left / n_samples) * impurity_left + \
                                   (n_right / n_samples) * impurity_right

                # Вычисляем уменьшение неоднородности
                impurity_decrease = parent_impurity - weighted_impurity

                # Проверяем минимальное уменьшение
                if impurity_decrease < self.min_impurity_decrease:
                    continue

                # Обновляем лучшее разделение
                if impurity_decrease > best_impurity_decrease:
                    best_impurity_decrease = impurity_decrease
                    best_feature = feature_idx
                    best_threshold = threshold

        return best_feature, best_threshold, best_impurity_decrease

    def _get_feature_indices(self, n_features):
        """Выбирает признаки для рассмотрения."""
        if self.max_features is None:
            return list(range(n_features))
        elif isinstance(self.max_features, int):
            return np.random.choice(n_features, self.max_features, replace=False).tolist()
        elif isinstance(self.max_features, float):
            n = max(1, int(self.max_features * n_features))
            return np.random.choice(n_features, n, replace=False).tolist()
        else:
            raise ValueError(f"Invalid max_features: {self.max_features}")

    def _build_tree(self, X, y, depth=0):
        """
        Рекурсивно строит дерево для регрессии.
        """
        n_samples, n_features = X.shape

        # Вычисляем среднее значение в узле (предсказание для листа)
        node_mean = np.mean(y) if len(y) > 0 else 0

        # Критерии остановки
        stop_conditions = [
            (self.max_depth is not None and depth >= self.max_depth),
            n_samples < self.min_samples_split,
            len(np.unique(y)) == 1,
            n_features == 0,
        ]

        if any(stop_conditions):
            # Создаем лист
            leaf_value = {
                'value': node_mean,
                'n_samples': n_samples,
                'mse': self._mse(y),
                'is_leaf': True
            }
            return leaf_value

        # Выбираем признаки
        feature_indices = self._get_feature_indices(n_features)

        # Ищем лучшее разделение
        best_feature, best_threshold, impurity_decrease = self._find_best_split(
            X, y, feature_indices
        )

        # Если не нашли подходящего разделения
        if best_feature is None or impurity_decrease <= 0:
            leaf_value = {
                'value': node_mean,
                'n_samples': n_samples,
                'mse': self._mse(y),
                'is_leaf': True
            }
            return leaf_value

        # Разделяем данные
        left_mask = X[:, best_feature] <= best_threshold
        right_mask = ~left_mask

        # Рекурсивно строим поддеревья
        left_subtree = self._build_tree(X[left_mask], y[left_mask], depth + 1)
        right_subtree = self._build_tree(X[right_mask], y[right_mask], depth + 1)

        # Создаем узел
        node = {
            'feature_idx': best_feature,
            'threshold': best_threshold,
            'impurity_decrease': impurity_decrease,
            'n_samples': n_samples,
            'value': node_mean,
            'mse': self._mse(y),
            'left': left_subtree,
            'right': right_subtree,
            'is_leaf': False
        }

        return node

    def _compute_feature_importances(self, node):
        """Вычисляет важность признаков для регрессии."""
        if node is None:
            return

        if not node.get('is_leaf', True):
            feature_idx = node['feature_idx']
            importance = (node['n_samples'] / self.n_samples_total) * node['impurity_decrease']

            self.feature_importances_[feature_idx] += importance

            self._compute_feature_importances(node['left'])
            self._compute_feature_importances(node['right'])

    def fit(self, X, y):
        """Обучает модель регрессии."""
        X = np.array(X)
        y = np.array(y)

        self.n_samples_total = X.shape[0]
        self.n_features = X.shape[1]
        self.feature_importances_ = np.zeros(self.n_features)

        self.tree = self._build_tree(X, y)
        self._compute_feature_importances(self.tree)

        # Нормализуем важность признаков
        if np.sum(self.feature_importances_) > 0:
            self.feature_importances_ /= np.sum(self.feature_importances_)

        return self

    def _predict_one(self, x, node):
        """Предсказывает значение для одного образца."""
        if node.get('is_leaf', True):
            return node['value']

        if x[node['feature_idx']] <= node['threshold']:
            return self._predict_one(x, node['left'])
        else:
            return self._predict_one(x, node['right'])

    def predict(self, X):
        """Предсказывает значения для набора данных."""
        X = np.array(X)
        n_samples = X.shape[0]
        predictions = np.zeros(n_samples)

        for i in range(n_samples):
            predictions[i] = self._predict_one(X[i], self.tree)

        return predictions

    def get_depth(self, node=None):
        """Возвращает глубину дерева."""
        if node is None:
            node = self.tree

        if node.get('is_leaf', True):
            return 0

        left_depth = self.get_depth(node['left'])
        right_depth = self.get_depth(node['right'])

        return max(left_depth, right_depth) + 1

    def get_n_leaves(self, node=None):
        """Возвращает количество листьев."""
        if node is None:
            node = self.tree

        if node.get('is_leaf', True):
            return 1

        left_leaves = self.get_n_leaves(node['left'])
        right_leaves = self.get_n_leaves(node['right'])

        return left_leaves + right_leaves

# Тестируем детальную реализацию для регрессии
print("\nТЕСТИРОВАНИЕ РЕАЛИЗАЦИИ ДЛЯ РЕГРЕССИИ")
print("-" * 60)

# Используем все числовые признаки
numeric_cols = ['Inches', 'Ram', 'Weight', 'ScreenW', 'ScreenH',
                'CPU_freq', 'PrimaryStorage', 'SecondaryStorage']

# Создаем модель
detailed_tree_reg = DetailedDecisionTreeRegressor(
    criterion='mse',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

# Обучаем
detailed_tree_reg.fit(X_train_l[numeric_cols].values, y_train_l.values)

# Предсказания
y_pred_detailed_reg = detailed_tree_reg.predict(X_test_l[numeric_cols].values)

# Оценка качества
mae_detailed_reg = mean_absolute_error(y_test_l, y_pred_detailed_reg)
mse_detailed_reg = mean_squared_error(y_test_l, y_pred_detailed_reg)
rmse_detailed_reg = np.sqrt(mse_detailed_reg)
r2_detailed_reg = r2_score(y_test_l, y_pred_detailed_reg)

print("Качество детальной реализации Decision Tree Regressor:")
print(f"MAE:   {mae_detailed_reg:.2f}")
print(f"MSE:   {mse_detailed_reg:.2f}")
print(f"RMSE:  {rmse_detailed_reg:.2f}")
print(f"R²:    {r2_detailed_reg:.4f}")

# Дополнительная информация
print(f"\nХарактеристики дерева:")
print(f"Глубина: {detailed_tree_reg.get_depth()}")
print(f"Количество листьев: {detailed_tree_reg.get_n_leaves()}")

# Важность признаков
print(f"\nТоп-5 самых важных признаков:")
feature_importance_reg_df = pd.DataFrame({
    'Признак': numeric_cols,
    'Важность': detailed_tree_reg.feature_importances_
}).sort_values('Важность', ascending=False)

display(feature_importance_reg_df.head(5))


ТЕСТИРОВАНИЕ РЕАЛИЗАЦИИ ДЛЯ РЕГРЕССИИ
------------------------------------------------------------
Качество детальной реализации Decision Tree Regressor:
MAE:   246.50
MSE:   120166.72
RMSE:  346.65
R²:    0.7579

Характеристики дерева:
Глубина: 5
Количество листьев: 32

Топ-5 самых важных признаков:


,Признак,Важность
1,Ram,0.740623
2,Weight,0.107863
5,CPU_freq,0.079405
3,ScreenW,0.032188
0,Inches,0.027387


Применение техник улучшенного бейзлайна к имплементированным моделям

In [92]:
# Улучшенная модель для классификации

improved_detailed_clf = DetailedDecisionTreeClassifier(
    criterion='gini',
    max_depth=7,  # Лучший параметр из GridSearch
    min_samples_split=2,
    min_samples_leaf=1,  # Лучший параметр из GridSearch
    min_impurity_decrease=0.0,
    random_state=42
)

improved_detailed_clf.fit(X_train_c.values, y_train_c)
y_pred_improved_detailed_clf = improved_detailed_clf.predict(X_test_c.values)

accuracy_improved_detailed_clf = accuracy_score(y_test_c, y_pred_improved_detailed_clf)
precision_improved_detailed_clf = precision_score(y_test_c, y_pred_improved_detailed_clf)
recall_improved_detailed_clf = recall_score(y_test_c, y_pred_improved_detailed_clf)
f1_improved_detailed_clf = f1_score(y_test_c, y_pred_improved_detailed_clf)

print(f"Accuracy:  {accuracy_improved_detailed_clf:.4f}")
print(f"Precision: {precision_improved_detailed_clf:.4f}")
print(f"Recall:    {recall_improved_detailed_clf:.4f}")
print(f"F1-score:  {f1_improved_detailed_clf:.4f}")
print(f"Глубина:   {improved_detailed_clf.get_depth()}")
print(f"Листья:    {improved_detailed_clf.get_n_leaves()}")

# Подготовка данных с логарифмированием
y_train_l_log = np.log1p(y_train_l.values)

improved_detailed_reg = DetailedDecisionTreeRegressor(
    criterion='mse',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

# Улучшенная модель для регрессии
print("Регрессия")

# Обучаем на логарифмированных данных
improved_detailed_reg.fit(X_train_l[numeric_cols].values, y_train_l_log)

# Предсказания
y_pred_log_improved = improved_detailed_reg.predict(X_test_l[numeric_cols].values)

# Обратное преобразование
y_pred_improved_detailed_reg = np.expm1(y_pred_log_improved)

# Оценка качества
mae_improved_detailed_reg = mean_absolute_error(y_test_l, y_pred_improved_detailed_reg)
rmse_improved_detailed_reg = np.sqrt(mean_squared_error(y_test_l, y_pred_improved_detailed_reg))
r2_improved_detailed_reg = r2_score(y_test_l, y_pred_improved_detailed_reg)

print(f"MAE:   {mae_improved_detailed_reg:.2f}")
print(f"RMSE:  {rmse_improved_detailed_reg:.2f}")
print(f"R²:    {r2_improved_detailed_reg:.4f}")
print(f"Глубина: {improved_detailed_reg.get_depth()}")
print(f"Листья:  {improved_detailed_reg.get_n_leaves()}")

Accuracy:  0.9386
Precision: 0.9487
Recall:    0.8810
F1-score:  0.9136
Глубина:   7
Листья:    22
Регрессия
MAE:   262.98
RMSE:  381.03
R²:    0.7075
Глубина: 5
Листья:  32


Сравнение улучшенных имплементированных моделей с улучшенным sklearn бейзлайном

In [95]:
print("КЛАССИФИКАЦИЯ:")
print(f"Accuracy:  {accuracy_improved_detailed_clf:.4f}")
print(f"Precision: {precision_improved_detailed_clf:.4f}")
print(f"Recall:    {recall_improved_detailed_clf:.4f}")
print(f"F1-score:  {f1_improved_detailed_clf:.4f}")
print(f"Глубина:   {improved_detailed_clf.get_depth()}")
print(f"Листья:    {improved_detailed_clf.get_n_leaves()}")

print("\nРЕГРЕССИЯ:")
print(f"MAE:   {mae_improved_detailed_reg:.2f}")
print(f"RMSE:  {rmse_improved_detailed_reg:.2f}")
print(f"R²:    {r2_improved_detailed_reg:.4f}")
print(f"Глубина: {improved_detailed_reg.get_depth()}")
print(f"Листья:  {improved_detailed_reg.get_n_leaves()}")

print("2. ОЦЕНКА КАЧЕСТВА РЕЗУЛЬТАТОВ")

# Создаем таблицу сравнения только детальной реализации и sklearn
final_comparison = pd.DataFrame({
    'Задача': ['Классификация'] * 3 + ['Регрессия'] * 3,
    'Модель': ['sklearn бейзлайн', 'sklearn улучшенная', 'Детальная улучшенная',
               'sklearn бейзлайн', 'sklearn улучшенная (лог.)', 'Детальная улучшенная (лог.)'],
    'Accuracy/MAE': [accuracy_base, accuracy_score(y_test_c, y_pred_c_best), accuracy_improved_detailed_clf,
                     mae_base, 203.07, mae_improved_detailed_reg],
    'F1/R²': [f1_base, f1_score(y_test_c, y_pred_c_best), f1_improved_detailed_clf,
              r2_base, 0.8255, r2_improved_detailed_reg],
    'Отставание от sklearn улучшенной': [
        f'{((accuracy_base - accuracy_score(y_test_c, y_pred_c_best))/accuracy_score(y_test_c, y_pred_c_best))*100:.1f}%',
        '0% (эталон)',
        f'{((accuracy_improved_detailed_clf - accuracy_score(y_test_c, y_pred_c_best))/accuracy_score(y_test_c, y_pred_c_best))*100:.1f}%',
        f'{((mae_base - 203.07)/203.07)*100:.1f}%',
        '0% (эталон)',
        f'{((mae_improved_detailed_reg - 203.07)/203.07)*100:.1f}%'
    ],
    'Оценка': [
        'Хороший бейзлайн',
        'Лучший результат',
        'Очень хорошо',
        'Хороший бейзлайн',
        'Лучший результат',
        'Удовлетворительно'
    ]
})

print("Сравнительная таблица (только детальная реализация vs sklearn):")
display(final_comparison)

print("3. ПОДРОБНАЯ ОЦЕНКА РЕЗУЛЬТАТОВ")

print("\n АНАЛИЗ КЛАССИФИКАЦИИ (Рак молочной железы):")

print(f"Детальная реализация показала Accuracy = {accuracy_improved_detailed_clf:.4f}")
print(f"По сравнению с sklearn улучшенной ({accuracy_score(y_test_c, y_pred_c_best):.4f}):")

accuracy_diff = accuracy_improved_detailed_clf - accuracy_score(y_test_c, y_pred_c_best)
if accuracy_diff > 0:
    print(f" ПРЕВОСХОДИТ на {abs(accuracy_diff)*100:.2f}%")
else:
    print(f"  Отстаёт на {abs(accuracy_diff)*100:.2f}%")

print(f"\nАнализ метрик классификации:")
print(f"• Precision: {precision_improved_detailed_clf:.4f} - очень хорошая точность")
print(f"• Recall: {recall_improved_detailed_clf:.4f} - неплохая полнота")
print(f"• F1-score: {f1_improved_detailed_clf:.4f} - хороший баланс точности и полноты")

print(f"\nХарактеристики дерева:")
print(f"• Глубина: {improved_detailed_clf.get_depth()} (оптимальная, нет переобучения)")
print(f"• Листья: {improved_detailed_clf.get_n_leaves()} (разумная сложность)")

print("\n АНАЛИЗ РЕГРЕССИИ (Цены ноутбуков):")

print(f"Детальная реализация показала MAE = {mae_improved_detailed_reg:.2f} евро, R² = {r2_improved_detailed_reg:.4f}")
print(f"По сравнению с sklearn улучшенной (MAE = 203.07 евро, R² = 0.8255):")

mae_diff_pct = ((mae_improved_detailed_reg - 203.07) / 203.07) * 100
r2_diff = r2_improved_detailed_reg - 0.8255

print(f"• MAE: хуже на {mae_diff_pct:.1f}%")
print(f"• R²: хуже на {abs(r2_diff)*100:.1f}%")

print(f"\nИнтерпретация метрик регрессии:")
print(f"• MAE = {mae_improved_detailed_reg:.2f} евро: средняя ошибка предсказания")
print(f"  → Модель ошибается в среднем на ±{mae_improved_detailed_reg:.0f} евро")
print(f"• R² = {r2_improved_detailed_reg:.4f}: объясняет {r2_improved_detailed_reg*100:.1f}% дисперсии")
print(f"  → Хорошая объясняющая способность")

print(f"\nХарактеристики дерева:")
print(f"• Глубина: {improved_detailed_reg.get_depth()} (разумная глубина)")
print(f"• Листья: {improved_detailed_reg.get_n_leaves()} (адекватная сложность)")

print("4. ВЫВОДЫ И ОЦЕНКА РЕЗУЛЬТАТОВ")

print("РЕКОМЕНДАЦИИ ДЛЯ УЛУЧШЕНИЯ")

print("\nДля улучшения качества регрессии:")
print("1. Реализовать более эффективный поиск лучшего разделения")
print("2. Добавить поддержку категориальных признаков")
print("3. Реализовать дополнительные критерии разделения")
print("4. Добавить пост-обрезку дерева (post-pruning)")

print("\n Детальная реализация решающего дерева УСПЕШНА!")
print("\nОсновные достижения:")
print(f"1. Классификация: Accuracy = {accuracy_improved_detailed_clf:.4f} (как у sklearn)")
print(f"2. Регрессия: R² = {r2_improved_detailed_reg:.4f} (хороший результат)")
print("3. Реализованы все ключевые компоненты алгоритма")
print("4. Техники улучшения эффективно работают")
print("5. Модели интерпретируемы и имеют разумную сложность")


КЛАССИФИКАЦИЯ:
Accuracy:  0.9386
Precision: 0.9487
Recall:    0.8810
F1-score:  0.9136
Глубина:   7
Листья:    22

РЕГРЕССИЯ:
MAE:   262.98
RMSE:  381.03
R²:    0.7075
Глубина: 5
Листья:  32
2. ОЦЕНКА КАЧЕСТВА РЕЗУЛЬТАТОВ
Сравнительная таблица (только детальная реализация vs sklearn):


,Задача,Модель,Accuracy/MAE,F1/R²,Отставание от sklearn улучшенной,Оценка
0,Классификация,sklearn бейзлайн,0.929825,0.904762,-0.9%,Хороший бейзлайн
1,Классификация,sklearn улучшенная,0.938596,0.913580,0% (эталон),Лучший результат
2,Классификация,Детальная улучшенная,0.938596,0.913580,0.0%,Очень хорошо
3,Регрессия,sklearn бейзлайн,234.667922,0.761442,15.6%,Хороший бейзлайн
4,Регрессия,sklearn улучшенная (лог.),203.070000,0.825500,0% (эталон),Лучший результат
5,Регрессия,Детальная улучшенная (лог.),262.975247,0.707495,29.5%,Удовлетворительно


3. ПОДРОБНАЯ ОЦЕНКА РЕЗУЛЬТАТОВ

 АНАЛИЗ КЛАССИФИКАЦИИ (Рак молочной железы):
Детальная реализация показала Accuracy = 0.9386
По сравнению с sklearn улучшенной (0.9386):
  Отстаёт на 0.00%

Анализ метрик классификации:
• Precision: 0.9487 - очень хорошая точность
• Recall: 0.8810 - неплохая полнота
• F1-score: 0.9136 - хороший баланс точности и полноты

Характеристики дерева:
• Глубина: 7 (оптимальная, нет переобучения)
• Листья: 22 (разумная сложность)

 АНАЛИЗ РЕГРЕССИИ (Цены ноутбуков):
Детальная реализация показала MAE = 262.98 евро, R² = 0.7075
По сравнению с sklearn улучшенной (MAE = 203.07 евро, R² = 0.8255):
• MAE: хуже на 29.5%
• R²: хуже на 11.8%

Интерпретация метрик регрессии:
• MAE = 262.98 евро: средняя ошибка предсказания
  → Модель ошибается в среднем на ±263 евро
• R² = 0.7075: объясняет 70.7% дисперсии
  → Хорошая объясняющая способность

Характеристики дерева:
• Глубина: 5 (разумная глубина)
• Листья: 32 (адекватная сложность)
4. ВЫВОДЫ И ОЦЕНКА РЕЗУЛЬТАТОВ
РЕКОМЕНДА